# RegNet Inference (submission generation)

In [1]:
import os
import sys
from pathlib import Path
import joblib
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import numpy as np
from torchvision.models import regnet_y_800mf, RegNet_Y_800MF_Weights
import timm
import re
from torchaudio import functional as F_audio

In [2]:
## REUSE IN INFERENCE NOTEBOOK

custom_dataset_path = '/kaggle/input/birdclef2023-inference'
if os.path.exists(os.path.join(custom_dataset_path, 'utils.py')):
    sys.path.append(custom_dataset_path)
else:
    sys.path.append('..')
import utils

IS_IN_KAGGLE_ENV = utils.get_is_in_kaggle_env()

DATA_PATH = '/kaggle/input/birdclef-2023' if IS_IN_KAGGLE_ENV else '../data'
JOBLIB_PATH = custom_dataset_path if IS_IN_KAGGLE_ENV else './'

DEVICE = 'cpu'

AUDIO_LENGTH_S = 5
SAMPLE_RATE = 32_000

We are running code on Localhost


In [3]:
## REUSE IN INFERENCE NOTEBOOK

class BirdMelspecClf(nn.Module):
    def __init__(self, out_features, pretrained):
        super().__init__()
        
        # https://pytorch.org/vision/stable/models.html

        self.regnet = regnet_y_800mf(weights=RegNet_Y_800MF_Weights.DEFAULT) if pretrained else regnet_y_800mf()

        """
        Original:
        RegnetCNN(
        (regnet): RegNet(
            (stem): SimpleStemIN(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
        )"""
        self.regnet.stem = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
        )
        
        # Fine-tune the regnet classifier
        self.regnet.fc = nn.Linear(self.regnet.fc.in_features, out_features)

        self.softmax = nn.Softmax(dim=1)
 
    def forward(self, x):
        logits = self.regnet(x)
        probas = self.softmax(logits)

        return logits, probas


def get_model(out_features, device, pretrained=False, load_state_dict=True, state_dict_starts_with=f"{AUDIO_LENGTH_S}s_regnetY800MF_"):
    model = BirdMelspecClf(out_features=out_features, pretrained=pretrained)
    print(f"Loaded model {model.__class__.__name__} with {sum(p.numel() for p in model.parameters())} parameters, pretained={pretrained}")
    model.to(device)

    if not load_state_dict:
        return model

    model_files = [f for f in os.listdir(JOBLIB_PATH) if f.startswith(state_dict_starts_with) and f.endswith('.pt')]
    if len(model_files) == 0:
        print(f"No model starting with {state_dict_starts_with} found in {JOBLIB_PATH}")
        return model
    
    # Extract timestamp from the filenames and sort based on it
    model_files.sort(key=lambda x: int(re.findall(r'\d+', x)[-1]) if re.findall(r'\d+', x) else -1)

    # The latest model file is the last one in the sorted list
    latest_model_file = model_files[-1]
    model_path = os.path.join(JOBLIB_PATH, latest_model_file)
    model.load_state_dict(torch.load(model_path))
    print(f"Loaded model weights from {model_path}")
    model.to(device)

    return model


def get_label_encoder():
    label_encoder_path = os.path.join(JOBLIB_PATH, 'label_encoder.joblib')
    label_encoder = joblib.load(label_encoder_path)
    print(f"Loaded label encoder from {label_encoder_path}")
    return label_encoder

In [4]:
## REUSE IN INFERENCE NOTEBOOK

def resample(audio, current_sample_rate, desired_sample_rate=SAMPLE_RATE):
    resampler = torchaudio.transforms.Resample(orig_freq=current_sample_rate, new_freq=desired_sample_rate)
    resampled_audio = resampler(audio)
    return resampled_audio

def load_audio(audio_path, sample_rate=SAMPLE_RATE):
    audio, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        audio = resample(audio, sr, sample_rate)
    return audio

# Using librosa defaults for n_fft and hop_length
def get_melspec_transform(sample_rate=SAMPLE_RATE, n_fft=2048, hop_length=512, n_mels=128):
    return torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
    )

# Using librosa defaults for top_db
def get_melspec_db_transform(stype='power', top_db=80):
    return torchaudio.transforms.AmplitudeToDB(
        stype=stype,
        top_db=top_db
    )

# Copied from torchaudio/transforms/_transforms.py (to avoid converting to melspec twice)
dct_mat = F_audio.create_dct(128, 128, "ortho")
def get_mfcc_from_melspec(melspec):
    return torch.matmul(melspec.transpose(-1, -2), dct_mat).transpose(-1, -2)

def normalize_tensor(tensor):
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    if max_val - min_val == 0:
        return tensor
    else:
        return (tensor - min_val) / (max_val - min_val)

In [5]:
filepaths = glob.glob(f"{DATA_PATH}/test_soundscapes/*.ogg")
print(f"filepaths length: {len(filepaths)} (amount of test soundscapes)")

filepaths length: 1 (amount of test soundscapes)


# Inference

In [6]:
debug = False
simulate_200_files = False

if simulate_200_files:
    filepaths = [filepaths[0] for i in range(200)] # simulate submission
    print(f"filepaths length: {len(filepaths)} after simulation additions")

label_encoder = get_label_encoder()
model = get_model(out_features=len(label_encoder.classes_), device=DEVICE, pretrained=False, load_state_dict=True)
model.eval()

MIN_WINDOW = AUDIO_LENGTH_S * SAMPLE_RATE
melspec_transform = get_melspec_transform(n_mels=128)
melspec_db_transform = get_melspec_db_transform()

def infer(filepath):
    all_predictions = []
    name = Path(filepath).stem
    audio = load_audio(filepath)
    audio_len_s = audio.shape[1] / SAMPLE_RATE
    debug and print(f"Infering file {filepath} with length {audio_len_s} s")
    n_crops = int(audio_len_s // 5)
    for i in range(n_crops):
        debug and print(f"Crop {i} / {n_crops}")
        debug and print(f"Audio length: {len(audio)}")
        crop = audio[:, i*MIN_WINDOW:(i+1)*MIN_WINDOW]
        debug and print(f"Crop dimensions: {crop.shape}")
        melspec = melspec_db_transform(melspec_transform(crop))
        norm_melspec = normalize_tensor(melspec)
        mfcc = get_mfcc_from_melspec(norm_melspec)
        norm_mfcc = normalize_tensor(mfcc)
        features = torch.cat((norm_melspec, norm_mfcc), dim=0)
        debug and print(f"features shape: {features.shape}") # [2, 128, 313]
        features = features.unsqueeze(0) # add batch dimension (1)
        debug and print(f"features unsqueezed shape: {features.shape}") # [1, 2, 128, 313]
        with torch.no_grad():
            logit, proba = model(features)
        t = (i + 1) * 5
        all_predictions.append({"row_id": f'{name}_{t}',"predictions": proba})
        debug and print('---')
    return all_predictions

if debug:
    all_preds = []
    for filepath in tqdm(filepaths, desc='Infering files'):
        all_preds.append(infer(filepath))
else:
    parallel_task = (delayed(infer)(filepath) for filepath in tqdm(filepaths, desc='Infering files'))
    all_preds = Parallel(n_jobs=os.cpu_count())(parallel_task)

all_preds_flat = [item for sublist in all_preds for item in sublist]

print(f"all_preds length: {len(all_preds)}, all_preds_flat length: {len(all_preds_flat)}")

Loaded label encoder from ./label_encoder.joblib
Loaded model BirdMelspecClf with 5854464 parameters, pretained=False
Loaded model weights from ./5s_regnetY800MF_e2_valacc73_traacc66_1684917323002.pt


Infering files: 100%|██████████| 1/1 [00:00<00:00, 141.80it/s]


all_preds length: 1, all_preds_flat length: 120


In [7]:
all_preds_flat[100]['predictions'][0]

tensor([0.0012, 0.0019, 0.0014, 0.0027, 0.0007, 0.0005, 0.0119, 0.0005, 0.0091,
        0.0004, 0.0022, 0.0005, 0.0008, 0.0046, 0.0033, 0.0006, 0.0016, 0.0018,
        0.0014, 0.0004, 0.0019, 0.0030, 0.0003, 0.0015, 0.0004, 0.0023, 0.0025,
        0.0023, 0.0216, 0.0062, 0.0021, 0.0014, 0.0003, 0.0023, 0.0003, 0.0002,
        0.0011, 0.0009, 0.0060, 0.0006, 0.0001, 0.0004, 0.0010, 0.0017, 0.0006,
        0.0022, 0.0027, 0.0004, 0.0004, 0.0005, 0.0004, 0.0011, 0.0013, 0.0007,
        0.0011, 0.0036, 0.1117, 0.0007, 0.0032, 0.0007, 0.0005, 0.0014, 0.0010,
        0.0021, 0.0002, 0.0004, 0.0772, 0.0004, 0.0002, 0.0013, 0.0005, 0.0015,
        0.0047, 0.0026, 0.0036, 0.0007, 0.0114, 0.0028, 0.0004, 0.0003, 0.0031,
        0.0005, 0.0006, 0.0004, 0.0010, 0.0003, 0.0017, 0.0099, 0.0060, 0.0032,
        0.0001, 0.0007, 0.0004, 0.0019, 0.0040, 0.0045, 0.0008, 0.0051, 0.0006,
        0.0018, 0.0003, 0.0011, 0.0004, 0.0053, 0.0085, 0.0010, 0.0009, 0.0006,
        0.0183, 0.0011, 0.0011, 0.0008, 

In [8]:
df = pd.concat([
    pd.DataFrame({'row_id': [p['row_id'] for p in all_preds_flat]}), 
    pd.DataFrame(torch.stack([p['predictions'][0] for p in all_preds_flat]).numpy(), columns=label_encoder.classes_)
], axis=1)

df

,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.010529,0.002436,0.010014,0.000580,0.000796,0.008274,0.001839,0.000450,0.007837,...,0.001445,0.000305,0.002611,0.000490,0.001268,0.000970,0.000401,0.000352,0.003227,0.001225
1,soundscape_29201_10,0.002820,0.002693,0.002539,0.005440,0.002211,0.001180,0.005649,0.001765,0.003575,...,0.001122,0.004476,0.006714,0.002493,0.001655,0.002523,0.004354,0.000888,0.001662,0.000965
2,soundscape_29201_15,0.004878,0.003323,0.004446,0.002952,0.001384,0.002717,0.002039,0.002309,0.007992,...,0.000978,0.002596,0.010041,0.001294,0.001384,0.001177,0.000745,0.002391,0.001050,0.000790
3,soundscape_29201_20,0.001254,0.001125,0.002086,0.014607,0.001837,0.001751,0.001378,0.001218,0.006614,...,0.000980,0.012608,0.004620,0.001473,0.000430,0.000973,0.002061,0.000542,0.001636,0.001005
4,soundscape_29201_25,0.001026,0.001234,0.002688,0.011050,0.002556,0.001456,0.002365,0.001535,0.006004,...,0.001264,0.008621,0.006365,0.002313,0.000465,0.002082,0.004618,0.000629,0.002567,0.001517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,soundscape_29201_580,0.000831,0.001897,0.001165,0.007363,0.001160,0.000630,0.003424,0.000648,0.005334,...,0.000396,0.010248,0.003067,0.000467,0.000437,0.001652,0.002259,0.000349,0.000721,0.000661
116,soundscape_29201_585,0.000363,0.000420,0.000084,0.000335,0.000118,0.000340,0.017743,0.000038,0.002064,...,0.000116,0.000514,0.000113,0.000022,0.000158,0.003326,0.004814,0.000014,0.000098,0.000190
117,soundscape_29201_590,0.001797,0.004096,0.001482,0.004525,0.001575,0.000529,0.025215,0.001719,0.003283,...,0.000948,0.007160,0.008161,0.001712,0.000694,0.005591,0.007531,0.000600,0.000960,0.000361
118,soundscape_29201_595,0.000891,0.000986,0.001564,0.006397,0.001979,0.000559,0.003133,0.001978,0.015307,...,0.000472,0.007305,0.003304,0.001038,0.000182,0.002797,0.015039,0.000479,0.001494,0.000593


In [9]:
df.to_csv('submission.csv', index=False)

In [10]:
!ls submission.csv

submission.csv
